In [6]:
import pandas as pd
# Load the data
data_df = pd.read_pickle('../data/parsed_data_all_positions.pkl')

data_dict = {
    'Age': [25, 30, 22],
    'Exp': [2, 5, 1],
    'G': [10, 15, 8],
    'Cmp': [5, 10, 4],
    # add other columns here
}
# Convert the dictionary to a pandas DataFrame
data_df = pd.DataFrame(data_dict)
# Now, you can check for missing values:
missing_values = data_df.isnull().sum()
print("Missing values per column:")
print(missing_values)
# Visualize the missing values with a heatmap
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
sns.heatmap(data_df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.show()

Missing values per column:
Age    0
Exp    0
G      0
Cmp    0
dtype: int64


ModuleNotFoundError: No module named 'seaborn'

In [4]:
# Check missing values across all columns
missing_values = data_df.isnull().sum()
print("Missing values per column:")
print(missing_values)
# You can also visualize missing values with a heatmap if you prefer
import matplotlib.pyplot as plt
# Heatmap to visualize missing values
plt.figure(figsize=(12, 8))
sns.heatmap(data_df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.show()
data_file='../data/parsed_data_all_positions.pkl'

data_df = data_df.apply(lambda col: col.fillna(col.mode()[0]) if col.dtypes == 'O' else col)

Missing values per column:
Age    0
Exp    0
G      0
Cmp    0
dtype: int64


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Visualizing outliers for some important numeric columns
import seaborn as sns
import matplotlib.pyplot as plt
# List of numeric columns you want to check
numeric_columns = [
    'Age', 'Exp', 'G', 'Cmp', 'Att', 'Cm%', 'PYd', 'Y/Att', 'PTD', 'Int',
    'Rsh', 'RshYd', 'RshTD', 'FP/G', 'FantPt', 'Y/Rsh', 'Rec', 'RecYd',
    'RecTD', 'Y/Rec', 'Year'
]

In [ ]:
# Set up the number of rows and columns dynamically to handle more than 20 subplots
num_columns = 4  # You can adjust this number as per your need
num_rows = (len(numeric_columns) + num_columns - 1) // num_columns  # Ensures enough rows for all plots
plt.figure(figsize=(15, num_rows * 3))  # Adjusting figure size based on number of rows
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(num_rows, num_columns, i)
    sns.boxplot(data_df[col])
    plt.title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
def generate_data(data_df: pd.DataFrame) -> Tuple[np.array, np.array]:
    '''Takes dataframe, input window size, parses data into feature label pairs,
    returns as tuple of numpy arrays'''
    # Get list of seasons
    seasons=data_df.index.get_level_values('Season').unique().tolist()
    features=[]
    labels=[]
    # Loop on seasons
    for season in seasons:
        # Extract the data for this season
        season_df=data_df.loc[(season),:,:]
        # Get the list of player for this season
        players=season_df.index.get_level_values('Player').unique().tolist()
        # loop on the players
        for player in players:
            # Extract the data for this player
            player_df=season_df.loc[:,(player),:]
            # Indexing variable for batch
            input_start_index=0
            # Loop on the player data
            while input_start_index + 1 < len(player_df):
                # Extract and collect the features and labels
                features.append(player_df.iloc[input_start_index].values.tolist())
                label_row=player_df.iloc[input_start_index + 1]
                labels.append(label_row['FantPt'])
                # Update the index
                input_start_index+=1
    return np.array(features), np.array(labels)

In [ ]:
# Compute the correlation matrix
correlation_matrix = data_df[numeric_columns].corr()
# Visualize the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()
New

In [ ]:
from sklearn.preprocessing import StandardScaler
# Apply Standard Scaler to numeric columns
scaler = StandardScaler()
data_df[numeric_columns] = scaler.fit_transform(data_df[numeric_columns])
# Check if scaling worked
print(data_df.head())